# Building user-based recommendation model for Amazon

## DESCRIPTION

The dataset provided contains movie reviews given by Amazon customers. Reviews were given between May 1996 and July 2014.

Data Dictionary

UserID – 4848 customers who provided a rating for each movie
Movie 1 to Movie 206 – 206 movies for which ratings are provided by 4848 distinct users

Data Considerations

- All the users have not watched all the movies and therefore, all movies are not rated. These missing values are represented by NA.
- Ratings are on a scale of -1 to 10 where -1 is the least rating and 10 is the best.

Analysis Task

- Exploratory Data Analysis:

Which movies have maximum views/ratings?
What is the average rating for each movie? Define the top 5 movies with the maximum ratings.
Define the top 5 movies with the least audience.
- Recommendation Model: Some of the movies hadn’t been watched and therefore, are not rated by the users. Netflix would like to take this as an opportunity and build a machine learning recommendation algorithm which provides the ratings for each of the users.

Divide the data into training and test data

Build a recommendation model on training data

Make predictions on the test data


In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import surprise

In [2]:
df = pd.read_csv('c:/HUAWEI/Desktop/PROJECT3//Amazon - Movies and TV Ratings.csv')

In [3]:
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(4848, 207)

In [5]:
df_org = df.copy()

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Movie1,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie2,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie3,1.0,2.000000,NaN,2.0,2.00,2.0,2.0,2.0
Movie4,2.0,5.000000,0.000000,5.0,5.00,5.0,5.0,5.0
Movie5,29.0,4.103448,1.496301,1.0,4.00,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
Movie202,6.0,4.333333,1.632993,1.0,5.00,5.0,5.0,5.0
Movie203,1.0,3.000000,NaN,3.0,3.00,3.0,3.0,3.0
Movie204,8.0,4.375000,1.407886,1.0,4.75,5.0,5.0,5.0
Movie205,35.0,4.628571,0.910259,1.0,5.00,5.0,5.0,5.0


## 1  - Which movies have maximum views/ratings?

In [7]:
# Movie with highest views

df.describe().T['count'].sort_values(ascending=False)[:1].to_frame() #---Movie 127

,count
Movie127,2313.0


In [8]:
# Movie with highest Ratings

df.drop('user_id',axis=1).sum().sort_values(ascending=False)[:1].to_frame()  #---Movie127

,0
Movie127,9511.0


## 2 - What is the average rating for each movie? Define the top 5 movies with the maximum ratings

In [9]:
df.drop('user_id',axis=1).mean().sort_values(ascending=False)[:5].to_frame()

,0
Movie1,5.0
Movie55,5.0
Movie131,5.0
Movie132,5.0
Movie133,5.0


## 3 - Define the top 5 movies with the least audience

In [10]:
df.describe().T['count'].sort_values(ascending=True)[:5].to_frame()

,count
Movie1,1.0
Movie71,1.0
Movie145,1.0
Movie69,1.0
Movie68,1.0


## 4 - Recommendation Model

In [11]:
from surprise import Reader
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate

In [12]:
df_melt = df.melt(id_vars = df.columns[0],value_vars=df.columns[1:],var_name="Movies",value_name="Rating")

In [13]:
df_melt

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [14]:
rd = Reader()
data = Dataset.load_from_df(df_melt.fillna(0),reader=rd)
data

In [15]:
trainset, testset = train_test_split(data,test_size=0.25)

In [16]:
# Using SVD (Singular Value Descomposition)

svd = SVD()
svd.fit(trainset)

In [17]:
pred = svd.test(testset)

In [18]:
accuracy.rmse(pred)

RMSE: 1.0255


1.0255256719106642

In [19]:
accuracy.mae(pred)

MAE:  1.0119


1.0118554100182764

In [20]:
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0263  1.0257  1.0261  1.0261  0.0002  
MAE (testset)     1.0121  1.0119  1.0121  1.0120  0.0001  
Fit time          25.68   25.80   25.78   25.75   0.05    
Test time         1.95    2.18    1.92    2.01    0.12    


{'test_rmse': array([1.02631625, 1.02575   , 1.02614003]),
 'test_mae': array([1.01206408, 1.01192111, 1.01214277]),
 'fit_time': (25.684950590133667, 25.795756101608276, 25.7755184173584),
 'test_time': (1.9476354122161865, 2.179455518722534, 1.9168214797973633)}

In [21]:
def repeat(ml_type,dframe):
    rd = Reader()
    data = Dataset.load_from_df(dframe,reader=rd)
    print(cross_validate(ml_type, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True))
    print("--"*15)
    usr_id = 'A3R5OBKS7OM2IR'
    mv = 'Movie1'
    r_u = 5.0
    print(ml_type.predict(usr_id,mv,r_ui = r_u,verbose=True))
    print("--"*15)


In [22]:
repeat(SVD(),df_melt.fillna(df_melt['Rating'].mean()))

# repeat(SVD(),df_melt.fillna(df_melt['Rating'].median()))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0896  0.0823  0.0859  0.0860  0.0030  
MAE (testset)     0.0099  0.0096  0.0098  0.0098  0.0001  
Fit time          25.63   25.81   26.45   25.97   0.35    
Test time         2.37    2.03    2.49    2.30    0.19    
{'test_rmse': array([0.08964193, 0.08234064, 0.08592831]), 'test_mae': array([0.00986446, 0.00963371, 0.00978577]), 'fit_time': (25.634488105773926, 25.813264846801758, 26.449771881103516), 'test_time': (2.368894577026367, 2.033384084701538, 2.4886722564697266)}
------------------------------
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.40   {'was_impossible': False}
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.40   {'was_impossible': False}
------------------------------


In [23]:
# trying grid search and find optimum hyperparameter value for n_factors

from surprise.model_selection import GridSearchCV

In [24]:
param_grid = {'n_epochs':[20,30],
             'lr_all':[0.005,0.001],
             'n_factors':[50,100]}

In [25]:
gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
data1 = Dataset.load_from_df(df_melt.fillna(df_melt['Rating'].mean()),reader=rd)
gs.fit(data1)

In [26]:
gs.best_score

{'rmse': 0.08476727174913357, 'mae': 0.009012317021054548}

In [27]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.08476727174913357
{'n_epochs': 30, 'lr_all': 0.001, 'n_factors': 50}
